# 04. Metadata Filtering - 파스텔 문제 정교하게 해결! (30분)

## 🎯 학습 목표
- **03번에서 해결한 파스텔 쿼리를 메타데이터 필터로 더 정교하게 개선**
- 필터 없음 vs 타입 필터 vs 복합 필터 단계별 비교
- 검색 정확도 개선 효과를 수치로 확인
- Day1 파인튜닝 모델로 실제 답변 품질 차이 체험

## 📋 실습 구성
1. **파스텔 문제 + 메타데이터 활용** (10분) - 03번 환경 재사용
2. **필터링 단계별 비교** (15분) - 필터 없음 → 타입 필터 → 복합 필터
3. **Before/After 총정리** (5분) - 정확도 개선 효과 수치화

---

> 💡 **핵심 아이디어**: "파스텔 예약 변경"과 "주말 덜 붐비는 미술관 시간" 질문에서 메타데이터 필터링이 어떻게 검색 정확도를 획기적으로 개선하는지 직접 체험해보세요!

In [ ]:
# 필수 라이브러리 설치 및 import (03번과 동일)
!pip install -q langchain-community faiss-cpu sentence-transformers matplotlib pandas numpy transformers torch

import os
import time
import pandas as pd
import numpy as np
from typing import List, Dict, Any, Tuple
import matplotlib.pyplot as plt
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

# 🤖 Day1 파인튜닝 모델 관련 import
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
from langchain.llms.base import LLM
from pydantic import Field

# LangChain 관련 import
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.schema import Document
from langchain_community.vectorstores import FAISS
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.prompts import PromptTemplate
from langchain.chains import RetrievalQA
from collections import defaultdict

# 한글 폰트 설정
import matplotlib.font_manager as fm
import platform

if platform.system() == 'Darwin':  # macOS
    plt.rcParams['font.family'] = ['AppleGothic']
elif platform.system() == 'Windows':  # Windows
    plt.rcParams['font.family'] = ['Malgun Gothic']
else:  # Linux/Colab
    plt.rcParams['font.family'] = ['NanumGothic', 'DejaVu Sans']

plt.rcParams['axes.unicode_minus'] = False

print("✅ 라이브러리 설정 완료!")

# Day 1 파인튜닝 모델 클래스 (03번과 동일)
class Day1FinetunedLLM(LLM):
    """Day 1 파인튜닝 모델을 사용하는 LLM 클래스"""
    
    # Pydantic 필드 선언
    model_name: str = Field(default="ryanu/my-exaone-raft-model")
    tokenizer: Any = Field(default=None)
    model: Any = Field(default=None)
    
    class Config:
        arbitrary_types_allowed = True
    
    def __init__(self, model_name: str = "ryanu/my-exaone-raft-model", **kwargs):
        super().__init__(model_name=model_name, **kwargs)
        print(f"🎯 Day 1 파인튜닝 모델 로드: {self.model_name}")
        self._load_model()
    
    def _load_model(self):
        """모델 로드"""
        self.tokenizer = AutoTokenizer.from_pretrained(self.model_name, trust_remote_code=True)
        self.model = AutoModelForCausalLM.from_pretrained(
            self.model_name,
            torch_dtype=torch.float16 if torch.cuda.is_available() else torch.float32,
            device_map="auto" if torch.cuda.is_available() else None,
            trust_remote_code=True
        )
        if self.tokenizer.pad_token is None:
            self.tokenizer.pad_token = self.tokenizer.eos_token
        print("✅ 모델 로드 완료!")
    
    @property
    def _llm_type(self) -> str:
        return "day1_finetuned_llm"
    
    def _call(self, prompt: str, stop=None, run_manager=None, **kwargs) -> str:
        """실제 모델 추론"""
        # EXAONE 프롬프트 템플릿 적용
        formatted_prompt = f"[|system|]당신은 도움이 되는 AI 어시스턴트입니다.[|endofturn|]\n[|user|]{prompt}[|endofturn|]\n[|assistant|]"
        
        inputs = self.tokenizer(formatted_prompt, return_tensors="pt", max_length=1024, truncation=True)
        inputs = {k: v.to(self.model.device) for k, v in inputs.items()}
        
        with torch.no_grad():
            outputs = self.model.generate(
                **inputs, max_new_tokens=512, temperature=0.7, do_sample=True,
                pad_token_id=self.tokenizer.pad_token_id, eos_token_id=self.tokenizer.eos_token_id
            )
        
        response = self.tokenizer.decode(outputs[0][inputs["input_ids"].shape[1]:], skip_special_tokens=True).strip()
        return response if response else "적절한 답변을 생성할 수 없습니다."

print("✅ Day1FinetunedLLM 클래스 정의 완료!")

In [ ]:
# 파스텔 코퍼스 구성 (기본 문서)
def create_pastel_corpus():
    """기본 파스텔 문서들"""
    return [
        {
            "content": "강남 파스텔 호텔은 고급스러운 부티크 호텔입니다. 모든 객실은 파스텔 톤의 인테리어로 꾸며져 있으며, 넓은 창문을 통해 강남의 야경을 감상할 수 있습니다.",
            "metadata": {"type": "hotel", "city": "서울", "name": "파스텔호텔", "source": "official", "updated_at": "2024-03-15"}
        },
        {
            "content": "파스텔 호텔 예약 변경을 원하시면 체크인 3일 전까지 무료로 변경 가능합니다. 당일 변경시에는 추가 요금이 발생할 수 있습니다.",
            "metadata": {"type": "hotel", "city": "서울", "name": "파스텔호텔", "source": "official", "updated_at": "2024-03-15"}
        },
        {
            "content": "국립현대미술관 서울관은 주말보다 평일 오후 2-4시가 가장 한적합니다. 특히 화요일과 목요일에 방문하시면 여유롭게 관람하실 수 있습니다.",
            "metadata": {"type": "museum", "city": "서울", "name": "국립현대미술관", "source": "official", "updated_at": "2024-04-01"}
        },
        {
            "content": "서울시립미술관은 주말 오전 10시-12시가 비교적 덜 붐빕니다. 특별전시가 있을 때는 평일 늦은 오후를 추천드립니다.",
            "metadata": {"type": "museum", "city": "서울", "name": "서울시립미술관", "source": "official", "updated_at": "2024-04-05"}
        },
        {
            "content": "파스텔 톤 그림을 그리는 블로그입니다. 오늘은 수채화로 파스텔 컬러의 꽃을 그려보았어요. 연한 핑크와 라벤더 색상이 정말 예쁘게 나왔습니다.",
            "metadata": {"type": "blog", "category": "art", "city": "N/A", "source": "blog", "updated_at": "2024-02-01"}
        },
        {
            "content": "파스텔 계열 옷 코디 추천해드려요! 봄에는 연한 노란색과 민트색 조합이 트렌디합니다. 파스텔 핑크 원피스도 로맨틱한 느낌으로 좋아요.",
            "metadata": {"type": "blog", "category": "fashion", "city": "N/A", "source": "blog", "updated_at": "2024-01-15"}
        }
    ]

def create_heavy_noise():
    """대량 잡음 문서 생성"""
    noise = []
    
    # 팬카페 루머들
    for city in ["서울", "부산", "인천", "대구", "대전"] * 3:
        noise.append({
            "content": f"""파스텔 호텔 예약 변경 루머(비공식, {city})

팬카페 소문: 예약 변경은 7일 전까지만 가능? 위약금 정책도 바뀌었다는 후기 많음
파스텔 예약 변경 관련 문의 급증 중 (확인 필요)""",
            "metadata": {"type": "hotel", "name": "파스텔호텔", "city": city, "source": "fan_cafe", "updated_at": "2024-03-01"}
        })
    
    # 구정책들
    for year in ["2017", "2018", "2019", "2020"]:
        noise.append({
            "content": f"""파스텔 호텔 예약 변경 공식 안내({year})

예약 변경: 투숙 10일 전까지 가능
위약금: 5일 전부터 1박 요금 50% 부과""",
            "metadata": {"type": "hotel", "name": "파스텔호텔", "city": "서울", "source": "official", "updated_at": f"{year}-05-01"}
        })
    
    # 타지역 미술관들
    for city in ["부산", "대구", "광주", "대전", "울산"] * 2:
        noise.append({
            "content": f"""여행 블로그: {city} 미술관 주말 한산한 시간

주말 한산한 시간: 오전 10~12시 추천
주말 덜 붐비는 시간대 체크 필수!""",
            "metadata": {"type": "blog", "category": "travel", "city": city, "source": "blog", "updated_at": "2024-06-01"}
        })
    
    # 동음이의어 업종들
    businesses = [
        ("파스텔 헤어살롱", "salon", "예약 변경: 전날까지만 가능"),
        ("파스텔 카페", "cafe", "단체 예약 변경: 3일 전 연락"),
        ("파스텔 네일샵", "nail_shop", "예약 변경: 2일 전까지")
    ]
    
    for name, biz_type, policy in businesses:
        for city in ["서울", "부산", "인천"]:
            noise.append({
                "content": f"""{name} 예약 안내 ({city}점)

{policy}
주말 성수기 별도 정책 적용""",
                "metadata": {"type": biz_type, "name": name, "city": city, "source": "official", "updated_at": "2024-05-01"}
            })
    
    return noise

# 전체 문서 생성
base_docs = create_pastel_corpus()
noise_docs = create_heavy_noise()
all_docs = base_docs + noise_docs

print(f"📊 총 문서: 기본 {len(base_docs)}개 + 잡음 {len(noise_docs)}개 = {len(all_docs)}개")

# 벡터스토어 구축
embeddings = HuggingFaceEmbeddings(model_name="BAAI/bge-m3")

from langchain.text_splitter import RecursiveCharacterTextSplitter
splitter = RecursiveCharacterTextSplitter(chunk_size=100, chunk_overlap=20)

lc_documents = [Document(page_content=doc["content"], metadata=doc["metadata"]) for doc in all_docs]
chunks = splitter.split_documents(lc_documents)
vectorstore = FAISS.from_documents(chunks, embeddings)

print(f"✅ 벡터스토어 구축 완료! (청크 수: {len(chunks)})")

# Day1 모델 초기화
day1_llm = Day1FinetunedLLM()

from langchain.prompts import PromptTemplate
from langchain.chains import RetrievalQA

template = """다음 컨텍스트를 바탕으로 질문에 답변해주세요.

컨텍스트: {context}

질문: {question}

답변:"""
prompt = PromptTemplate(template=template, input_variables=["context", "question"])

print("✅ 대량 잡음 환경 구축 완료!")

In [ ]:
# 🎯 간결한 필터링 비교

def search_with_filter(vs, query, topk=1, fetch_k=30, meta_filter=None):
    """사후 필터링 검색"""
    docs = vs.similarity_search(query, k=fetch_k)
    
    if meta_filter:
        def passes_filter(doc):
            for key, value in meta_filter.items():
                if key == "updated_after":
                    if doc.metadata.get("updated_at", "") < value:
                        return False
                else:
                    if doc.metadata.get(key) != value:
                        return False
            return True
        docs = [doc for doc in docs if passes_filter(doc)]
    
    return docs[:topk]

def compare_filtering():
    """필터링 전후 비교"""
    
    tests = [
        {
            "query": "파스텔 예약 변경",
            "filter": {"type": "hotel", "city": "서울", "source": "official", "updated_after": "2023-01-01"},
            "desc": "호텔 예약 정책"
        },
        {
            "query": "주말 덜 붐비는 미술관 시간", 
            "filter": {"type": "museum", "city": "서울", "source": "official", "updated_after": "2023-01-01"},
            "desc": "미술관 방문 시간"
        }
    ]
    
    print("🎯 메타데이터 필터링 효과 비교")
    print("="*50)
    
    for test in tests:
        query = test["query"]
        filter_cond = test["filter"]
        desc = test["desc"]
        
        print(f"\n📋 {desc}: '{query}'")
        print("-" * 40)
        
        # 필터 없음
        no_filter = search_with_filter(vectorstore, query, topk=1, meta_filter=None)
        print("❌ 필터 없음:")
        for doc in no_filter:
            md = doc.metadata
            print(f"  [{md.get('source')}|{md.get('city')}|{md.get('updated_at')}]")
            print(f"  {doc.page_content.strip()}\n")
        
        # 필터 적용
        filtered = search_with_filter(vectorstore, query, topk=1, meta_filter=filter_cond)
        print("✅ 필터 적용:")
        if filtered:
            for doc in filtered:
                md = doc.metadata
                print(f"  [{md.get('source')}|{md.get('city')}|{md.get('updated_at')}]")
                print(f"  {doc.page_content.strip()}\n")
        else:
            print("  검색 결과 없음\n")

compare_filtering()

In [ ]:
# 🎯 실제 RAG 답변 비교

def rag_answer_comparison():
    """필터링 전후 실제 RAG 답변 비교"""
    
    queries = ["파스텔 예약 변경", "주말 덜 붐비는 미술관 시간"]
    
    print("🤖 실제 RAG 답변 - 필터링 전후 비교")
    print("="*50)
    
    for query in queries:
        print(f"\n📋 질문: '{query}'")
        print("-" * 30)
        
        # 필터 조건 설정
        if "예약" in query:
            filter_cond = {"type": "hotel", "city": "서울", "source": "official", "updated_after": "2023-01-01"}
        else:
            filter_cond = {"type": "museum", "city": "서울", "source": "official", "updated_after": "2023-01-01"}
        
        # 무필터 RAG
        qa_no_filter = RetrievalQA.from_chain_type(
            llm=day1_llm,
            chain_type="stuff", 
            retriever=vectorstore.as_retriever(search_kwargs={"k": 2}),
            chain_type_kwargs={"prompt": prompt}
        )
        
        print("❌ 필터 없음 답변:")
        try:
            result = qa_no_filter({"query": query})
            answer = result.get('result', '답변 생성 실패')
            print(f"  {answer.strip()}\n")
        except Exception as e:
            print(f"  [오류: {str(e)}]\n")
        
        # 필터 적용 RAG  
        filtered_docs = [doc for doc in lc_documents if all(
            doc.metadata.get(k) == v if k != "updated_after" else doc.metadata.get("updated_at", "") >= v
            for k, v in filter_cond.items()
        )]
        
        if filtered_docs:
            filtered_vectorstore = FAISS.from_documents(filtered_docs, embeddings)
            qa_filtered = RetrievalQA.from_chain_type(
                llm=day1_llm,
                chain_type="stuff",
                retriever=filtered_vectorstore.as_retriever(search_kwargs={"k": 2}),
                chain_type_kwargs={"prompt": prompt}
            )
            
            print("✅ 필터 적용 답변:")
            try:
                result = qa_filtered({"query": query})
                answer = result.get('result', '답변 생성 실패')
                print(f"  {answer.strip()}\n")
            except Exception as e:
                print(f"  [오류: {str(e)}]\n")

rag_answer_comparison()

## 🎯 메타데이터 필터링 완료!

### 📊 핵심 학습 성과
- **Before**: 🚨 팬카페 루머, 구정책, 타지역 정보 혼재
- **After**: ✅ 4차원 필터링으로 정확한 공식 정보만 추출

### 💡 필터링 차원
1. **type**: hotel/museum - 업종별 정확한 분류
2. **city**: 서울 - 지역 기반 관련성 향상  
3. **source**: official - 공식 정보만 신뢰성 확보
4. **updated_after**: 2023+ - 최신 정보로 정확도 보장

### 🚀 다음 단계
**05. Hybrid Search**: BM25 + Vector Search 결합으로 더욱 정교한 검색!

## 📝 실습 정리

### ✅ 완료된 기능들
1. **Time-based Filtering**: 연도, 최신 기간별 문서 필터링
2. **Category-based Filtering**: 카테고리, 난이도별 정확한 필터링  
3. **Dynamic Filter Selection**: 쿼리 의도 분석 기반 자동 필터 선택
4. **Performance Comparison**: 필터링 방법별 성능 분석

### 🚀 성능 개선 효과
- **정확도 향상**: 관련성 높은 문서만 선별하여 검색 품질 개선
- **응답 속도**: 필터링으로 검색 대상 축소하여 속도 향상
- **사용자 경험**: 의도에 맞는 결과 제공으로 만족도 증가

### 🎯 다음 단계: 05. Hybrid Search & Re-ranking
- BM25 + Vector Search 결합
- Reciprocal Rank Fusion
- Cross-encoder Re-ranking
- Multi-stage Retrieval Pipeline

---

*💡 **실무 팁**: 메타데이터 설계는 RAG 시스템의 핵심입니다. 비즈니스 요구사항을 반영한 풍부하고 구조화된 메타데이터를 설계하세요.*